In [1]:
import dlib, cv2
import numpy as np

In [2]:
detector = dlib.get_frontal_face_detector() #얼굴 검출
sp = dlib.shape_predictor('models/shape_predictor_68_face_landmarks.dat') #얼굴 68개 랜드마크 검출
facerec = dlib.face_recognition_model_v1('models/dlib_face_recognition_resnet_model_v1.dat') #얼굴 인식

In [3]:
descs_minsu= np.load('img/descs_minsu.npy',allow_pickle=True) [()]

In [4]:
cap = cv2.VideoCapture(0)
cap.set(3,640)
cap.set(4,480)

if not cap.isOpened():
    exit()

In [5]:
def encode_face(img):
    dets = detector(img, 1)

    if len(dets) == 0:
        return np.empty(0)

    for k, d in enumerate(dets):
        shape = sp(img, d)
        face_descriptor = facerec.compute_face_descriptor(img, shape)

    return np.array(face_descriptor)

In [6]:
while True:
    ret, frame = cap.read()
    
    if not ret:
        break
        
    #img_rgb = cv2.cvtColor(frame, ret)
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    dets = detector(frame, 1)
    
    for k, d in enumerate(dets):
        shape = sp(frame, d)
        face_descriptor = facerec.compute_face_descriptor(img_rgb, shape)

        last_found = {'name': 'unknown', 'dist': 0.6, 'color': (0,0,255)}

        for name, saved_desc in descs_minsu.items():
            dist = np.linalg.norm([face_descriptor] - saved_desc, axis=1)

            if dist < last_found['dist']:
                last_found = {'name': name, 'dist': dist, 'color': (255,255,255)}

        cv2.rectangle(frame, pt1=(d.left(), d.top()), pt2=(d.right(), d.bottom()), 
                  color=last_found['color'], thickness=2)
    
        cv2.putText(frame, last_found['name'], org=(d.left(), d.top()), fontFace=cv2.FONT_HERSHEY_SIMPLEX, 
                fontScale=1, color=last_found['color'], thickness=2)
    
    cv2.imshow('face', frame)
    
    if(cv2.waitKey(1)&0xFF ==ord('q')) :
        break
        
cap.release()
cv2.destroyAllWindows()